In [1]:
import dash
from dash import dcc, html, Input, Output
import plotly.graph_objs as go
import yfinance as yf
import pandas as pd
import ta

# Initialisation de l'application Dash
app = dash.Dash(__name__)

# Actions sélectionnées
tickers = ['MC.PA', 'OR.PA', 'AIR.PA', 'SAN.PA', 'BNP.PA','GLE.PA', 'ENGI.PA', 'VIV.PA', 'SGO.PA', 'CA.PA','EURUSD=X', 'GBPUSD=X', 'BTC-USD', 'ETH-USD']

# Télécharger les données
data = {ticker: yf.download(ticker, start="2022-01-01", end="2024-11-26") for ticker in tickers}

# Prétraitement des données
for ticker, df in data.items():
    df['Close'] = df['Close'].fillna(method='ffill').fillna(method='bfill')
    df['High'] = df['High'].fillna(method='ffill').fillna(method='bfill')
    df['Low'] = df['Low'].fillna(method='ffill').fillna(method='bfill')
    close_series = df['Close'].squeeze()
    
    df['returns'] = df['Close'].pct_change()
    df['volatility'] = df['returns'].rolling(window=21).std()
    df['SMA_50'] = close_series.rolling(window=50).mean()
    df['RSI'] = ta.momentum.rsi(close_series, window=14)
    df['high_low'] = df['High'] - df['Low']
    df['high_close'] = abs(df['High'] - df['Close'].shift())
    df['low_close'] = abs(df['Low'] - df['Close'].shift())
    df['TR'] = df[['high_low', 'high_close', 'low_close']].max(axis=1)
    df['ATR'] = df['TR'].rolling(window=14).mean()

    # Calcul du drawdown
    df['cumulative_returns'] = (1 + df['returns']).cumprod()
    df['running_max'] = df['cumulative_returns'].cummax()
    df['drawdown'] = df['cumulative_returns'] / df['running_max'] - 1

# Layout du dashboard
app.layout = html.Div([
    html.H1("Dashboard d'Analyse des Actions", style={'textAlign': 'center'}),
    dcc.Dropdown(
        id='ticker-dropdown',
        options=[{'label': ticker, 'value': ticker} for ticker in tickers],
        value='MC.PA',
        style={'width': '50%', 'margin': 'auto'}
    ),
    dcc.Graph(id='sma50-chart'),
    dcc.Graph(id='atr-chart'),
    dcc.Graph(id='volatility-chart'),
    dcc.Graph(id='rsi-chart'),
    html.Div(id='metrics-cards', style={
        'display': 'flex',
        'justifyContent': 'space-around',
        'marginTop': '20px'
    }),
])

# Callbacks pour mettre à jour les graphiques et les métriques
@app.callback(
    [
     Output('sma50-chart', 'figure'),
     Output('atr-chart', 'figure'),
     Output('volatility-chart', 'figure'),
     Output('rsi-chart', 'figure'),
     Output('metrics-cards', 'children')],
    [Input('ticker-dropdown', 'value')]
)
def update_charts(ticker):
    df = data[ticker]


    # Graphique de la SMA50
    sma50_fig = go.Figure()
    sma50_fig.add_trace(go.Scatter(x=df.index, y=df['SMA_50'], mode='lines', name='SMA 50', line=dict(color='orange')))
    sma50_fig.update_layout(title=f'{ticker} - SMA 50', xaxis_title='Date', yaxis_title='SMA 50')

    # Graphique de l'ATR
    atr_fig = go.Figure()
    atr_fig.add_trace(go.Scatter(x=df.index, y=df['ATR'], mode='lines', name='ATR', line=dict(color='green')))
    atr_fig.update_layout(title=f'{ticker} - ATR', xaxis_title='Date', yaxis_title='ATR')

    # Graphique de la volatilité
    vol_fig = go.Figure()
    vol_fig.add_trace(go.Scatter(x=df.index, y=df['volatility'], mode='lines', name='Volatilité', line=dict(color='blue')))
    vol_fig.update_layout(title=f'{ticker} - Volatilité', xaxis_title='Date', yaxis_title='Volatilité')

    # Graphique du RSI
    rsi_fig = go.Figure()
    rsi_fig.add_trace(go.Scatter(x=df.index, y=df['RSI'], mode='lines', name='RSI', line=dict(color='purple')))
    rsi_fig.add_hline(y=30, line_dash="dot", annotation_text="Survendu (30)", line_color="red")
    rsi_fig.add_hline(y=70, line_dash="dot", annotation_text="Suracheté (70)", line_color="green")
    rsi_fig.update_layout(title=f'{ticker} - RSI', xaxis_title='Date', yaxis_title='RSI')

    # Calcul des métriques
    sharpe_ratio = df['returns'].mean() / df['returns'].std() * (252 ** 0.5)
    max_drawdown = df['drawdown'].min()
    atr_last = df['ATR'].iloc[-1]
    vol_last = df['volatility'].iloc[-1]

    # Création des cartes pour les métriques
    metrics_cards = [
        html.Div([
            html.H4("Sharpe Ratio", style={'textAlign': 'center'}),
            html.P(f"{sharpe_ratio:.2f}", style={'fontSize': '24px', 'textAlign': 'center', 'margin': '0'})
        ], style={
            'border': '1px solid #ccc', 'borderRadius': '10px', 'padding': '20px',
            'width': '20%', 'boxShadow': '2px 2px 5px rgba(0, 0, 0, 0.1)', 'backgroundColor': '#f9f9f9'
        }),
        html.Div([
            html.H4("Drawdown Max", style={'textAlign': 'center'}),
            html.P(f"{max_drawdown:.2%}", style={'fontSize': '24px', 'textAlign': 'center', 'margin': '0'})
        ], style={
            'border': '1px solid #ccc', 'borderRadius': '10px', 'padding': '20px',
            'width': '20%', 'boxShadow': '2px 2px 5px rgba(0, 0, 0, 0.1)', 'backgroundColor': '#f9f9f9'
        }),
        html.Div([
            html.H4("Volatilité", style={'textAlign': 'center'}),
            html.P(f"{vol_last:.2f}", style={'fontSize': '24px', 'textAlign': 'center', 'margin': '0'})
        ], style={
            'border': '1px solid #ccc', 'borderRadius': '10px', 'padding': '20px',
            'width': '20%', 'boxShadow': '2px 2px 5px rgba(0, 0, 0, 0.1)', 'backgroundColor': '#f9f9f9'
        }),
        html.Div([
            html.H4("ATR", style={'textAlign': 'center'}),
            html.P(f"{atr_last:.2f}", style={'fontSize': '24px', 'textAlign': 'center', 'margin': '0'})
        ], style={
            'border': '1px solid #ccc', 'borderRadius': '10px', 'padding': '20px',
            'width': '20%', 'boxShadow': '2px 2px 5px rgba(0, 0, 0, 0.1)', 'backgroundColor': '#f9f9f9'
        }),
    ]

    return sma50_fig, atr_fig, vol_fig, rsi_fig, metrics_cards

# Lancer l'application
if __name__ == '__main__':
    app.run_server(debug=True)


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.1.3 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "c:\Users\LENOVO\miniconda3(1)\lib\runpy.py", line 196, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "c:\Users\LENOVO\miniconda3(1)\lib\runpy.py", line 86, in _run_code
    exec(code, run_globals)
  File "C:\Users\LENOVO\AppData\Roaming\Python\Python310\site-packages\ipykernel_launcher.py", line 17, in <module>
    app.launch_new_instance()
  File "C:\Users\LENOVO\AppData\Roaming\Python\Python310\site-packages\traitlets\config\application.py", line 1043, in launch_instance
    app.sta

AttributeError: _ARRAY_API not found


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.1.3 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "c:\Users\LENOVO\miniconda3(1)\lib\runpy.py", line 196, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "c:\Users\LENOVO\miniconda3(1)\lib\runpy.py", line 86, in _run_code
    exec(code, run_globals)
  File "C:\Users\LENOVO\AppData\Roaming\Python\Python310\site-packages\ipykernel_launcher.py", line 17, in <module>
    app.launch_new_instance()
  File "C:\Users\LENOVO\AppData\Roaming\Python\Python310\site-packages\traitlets\config\application.py", line 1043, in launch_instance
    app.sta

AttributeError: _ARRAY_API not found

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
C:\Users\LENOVO\AppData\Local\Temp

In [4]:
import dash
from dash import dcc, html, Input, Output
import plotly.graph_objs as go
import yfinance as yf
import pandas as pd
import ta

# Initialisation de l'application Dash
app = dash.Dash(__name__)

# Actions sélectionnées
tickers = ['MC.PA', 'OR.PA', 'AIR.PA', 'SAN.PA', 'BNP.PA', 'GLE.PA', 'ENGI.PA', 'VIV.PA', 'SGO.PA', 'CA.PA', 'EURUSD=X', 'GBPUSD=X', 'BTC-USD', 'ETH-USD']

# Télécharger les données
data = {ticker: yf.download(ticker, start="2022-01-01", end="2024-11-26") for ticker in tickers}

# Prétraitement des données
for ticker, df in data.items():
    df['Close'] = df['Close'].fillna(method='ffill').fillna(method='bfill')
    df['High'] = df['High'].fillna(method='ffill').fillna(method='bfill')
    df['Low'] = df['Low'].fillna(method='ffill').fillna(method='bfill')
    close_series = df['Close'].squeeze()

    # Indicateurs techniques
    df['returns'] = df['Close'].pct_change()
    df['volatility'] = df['returns'].rolling(window=21).std()
    df['SMA_50'] = close_series.rolling(window=50).mean()
    df['RSI'] = ta.momentum.rsi(close_series, window=14)
    df['high_low'] = df['High'] - df['Low']
    df['high_close'] = abs(df['High'] - df['Close'].shift())
    df['low_close'] = abs(df['Low'] - df['Close'].shift())
    df['TR'] = df[['high_low', 'high_close', 'low_close']].max(axis=1)
    df['ATR'] = df['TR'].rolling(window=14).mean()

    # Bandes de Bollinger
    df['bollinger_mid'] = close_series.rolling(window=20).mean()
    df['bollinger_upper'] = df['bollinger_mid'] + 2 * close_series.rolling(window=20).std()
    df['bollinger_lower'] = df['bollinger_mid'] - 2 * close_series.rolling(window=20).std()

    # MACD
    df['MACD'] = ta.trend.macd(close_series, window_slow=26, window_fast=12)
    df['MACD_signal'] = ta.trend.macd_signal(close_series, window_slow=26, window_fast=12, window_sign=9)

    # Calcul du drawdown
    df['cumulative_returns'] = (1 + df['returns']).cumprod()
    df['running_max'] = df['cumulative_returns'].cummax()
    df['drawdown'] = df['cumulative_returns'] / df['running_max'] - 1

# Layout du dashboard
app.layout = html.Div([
    html.H1("Dashboard d'Analyse des Actions", style={'textAlign': 'center'}),
    dcc.Dropdown(
        id='ticker-dropdown',
        options=[{'label': ticker, 'value': ticker} for ticker in tickers],
        value='MC.PA',
        style={'width': '50%', 'margin': 'auto'}
    ),
    dcc.Graph(id='sma50-chart'),
    dcc.Graph(id='atr-chart'),
    dcc.Graph(id='volatility-chart'),
    dcc.Graph(id='rsi-chart'),
    dcc.Graph(id='bollinger-chart'),
    dcc.Graph(id='macd-chart'),
    html.Div(id='metrics-cards', style={
        'display': 'flex',
        'justifyContent': 'space-around',
        'marginTop': '20px'
    }),
])

# Callbacks pour mettre à jour les graphiques et les métriques
@app.callback(
    [
     Output('sma50-chart', 'figure'),
     Output('atr-chart', 'figure'),
     Output('volatility-chart', 'figure'),
     Output('rsi-chart', 'figure'),
     Output('bollinger-chart', 'figure'),
     Output('macd-chart', 'figure'),
     Output('metrics-cards', 'children')],
    [Input('ticker-dropdown', 'value')]
)
def update_charts(ticker):
    df = data[ticker]

    # Graphique de la SMA50
    sma50_fig = go.Figure()
    sma50_fig.add_trace(go.Scatter(x=df.index, y=df['SMA_50'], mode='lines', name='SMA 50', line=dict(color='orange')))
    sma50_fig.update_layout(title=f'{ticker} - SMA 50', xaxis_title='Date', yaxis_title='SMA 50')

    # Graphique de l'ATR
    atr_fig = go.Figure()
    atr_fig.add_trace(go.Scatter(x=df.index, y=df['ATR'], mode='lines', name='ATR', line=dict(color='green')))
    atr_fig.update_layout(title=f'{ticker} - ATR', xaxis_title='Date', yaxis_title='ATR')

    # Graphique de la volatilité
    vol_fig = go.Figure()
    vol_fig.add_trace(go.Scatter(x=df.index, y=df['volatility'], mode='lines', name='Volatilité', line=dict(color='blue')))
    vol_fig.update_layout(title=f'{ticker} - Volatilité', xaxis_title='Date', yaxis_title='Volatilité')

    # Graphique du RSI
    rsi_fig = go.Figure()
    rsi_fig.add_trace(go.Scatter(x=df.index, y=df['RSI'], mode='lines', name='RSI', line=dict(color='purple')))
    rsi_fig.add_hline(y=30, line_dash="dot", annotation_text="Survendu (30)", line_color="red")
    rsi_fig.add_hline(y=70, line_dash="dot", annotation_text="Suracheté (70)", line_color="green")
    rsi_fig.update_layout(title=f'{ticker} - RSI', xaxis_title='Date', yaxis_title='RSI')

    # Graphique des Bandes de Bollinger
    bollinger_fig = go.Figure()
    bollinger_fig.add_trace(go.Scatter(x=df.index, y=df['Close'], mode='lines', name='Close', line=dict(color='blue')))
    bollinger_fig.add_trace(go.Scatter(x=df.index, y=df['bollinger_upper'], mode='lines', name='Bollinger Upper', line=dict(color='red')))
    bollinger_fig.add_trace(go.Scatter(x=df.index, y=df['bollinger_lower'], mode='lines', name='Bollinger Lower', line=dict(color='green')))
    bollinger_fig.update_layout(title=f'{ticker} - Bandes de Bollinger', xaxis_title='Date', yaxis_title='Prix')

    # Graphique du MACD
    macd_fig = go.Figure()
    macd_fig.add_trace(go.Scatter(x=df.index, y=df['MACD'], mode='lines', name='MACD', line=dict(color='orange')))
    macd_fig.add_trace(go.Scatter(x=df.index, y=df['MACD_signal'], mode='lines', name='MACD Signal', line=dict(color='blue')))
    macd_fig.update_layout(title=f'{ticker} - MACD', xaxis_title='Date', yaxis_title='MACD')

    # Calcul des métriques
    sharpe_ratio = df['returns'].mean() / df['returns'].std() * (252 ** 0.5)
    max_drawdown = df['drawdown'].min()
    atr_last = df['ATR'].iloc[-1]
    vol_last = df['volatility'].iloc[-1]
    bollinger_range = df['bollinger_upper'].iloc[-1] - df['bollinger_lower'].iloc[-1]
    macd_last = df['MACD'].iloc[-1]

    # Création des cartes pour les métriques
    metrics_cards = [
        html.Div([
            html.H4("Sharpe Ratio", style={'textAlign': 'center'}),
            html.P(f"{sharpe_ratio:.2f}", style={'fontSize': '24px', 'textAlign': 'center', 'margin': '0'})
        ], style={
            'border': '1px solid #ccc', 'borderRadius': '10px', 'padding': '20px',
            'width': '20%', 'boxShadow': '2px 2px 5px rgba(0, 0, 0, 0.1)', 'backgroundColor': '#f9f9f9'
        }),
        html.Div([
            html.H4("Drawdown Max", style={'textAlign': 'center'}),
            html.P(f"{max_drawdown:.2%}", style={'fontSize': '24px', 'textAlign': 'center', 'margin': '0'})
        ], style={
            'border': '1px solid #ccc', 'borderRadius': '10px', 'padding': '20px',
            'width': '20%', 'boxShadow': '2px 2px 5px rgba(0, 0, 0, 0.1)', 'backgroundColor': '#f9f9f9'
        }),
        html.Div([
            html.H4("Range Bollinger", style={'textAlign': 'center'}),
            html.P(f"{bollinger_range:.2f}", style={'fontSize': '24px', 'textAlign': 'center', 'margin': '0'})
        ], style={
            'border': '1px solid #ccc', 'borderRadius': '10px', 'padding': '20px',
            'width': '20%', 'boxShadow': '2px 2px 5px rgba(0, 0, 0, 0.1)', 'backgroundColor': '#f9f9f9'
        }),
        html.Div([
            html.H4("MACD Dernier", style={'textAlign': 'center'}),
            html.P(f"{macd_last:.2f}", style={'fontSize': '24px', 'textAlign': 'center', 'margin': '0'})
        ], style={
            'border': '1px solid #ccc', 'borderRadius': '10px', 'padding': '20px',
            'width': '20%', 'boxShadow': '2px 2px 5px rgba(0, 0, 0, 0.1)', 'backgroundColor': '#f9f9f9'
        }),
    ]

    return sma50_fig, atr_fig, vol_fig, rsi_fig, bollinger_fig, macd_fig, metrics_cards

# Lancer l'application
if __name__ == '__main__':
    app.run_server(debug=True)


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
C:\Users\LENOVO\AppData\Local\Temp